In [ ]:
import tensorflow as tf
from scipy.stats import gaussian_kde
import numpy as np

tf.enable_eager_execution()

g = tf.Graph()
with g.as_default():
    N = 10000
    Wafer1 = np.random.normal(loc = 0.0, scale = 1.0, size = N)
    Wafer2 = np.random.normal(loc = 2.0, scale = 1, size = N)
    
    pdf1 = gaussian_kde(Wafer1)
    pdf2 = gaussian_kde(Wafer2)
    
    pdf1.set_bandwidth(bw_method=pdf1.factor / 3.)
    pdf2.set_bandwidth(bw_method=pdf2.factor / 3.)

    def lin_model(x, w, b):
        return tf.add(tf.scalar_mul(w, x), b)
    
    def error_function(prediction, teacher):
#         print(np.shape(prediction), np.shape(teacher))
        return tf.reduce_mean(tf.square(prediction-teacher))

    def pdf_wrapper(y):
        temp = y.numpy()
        result = pdf2(temp)
        result = tf.convert_to_tensor(result, dtype=tf.float32)
        return result
    
    def generate_grid_data():
        w_min, w_max = -3, 3
        b_min, b_max = -3, 3
        
        ww, bb = np.mgrid[w_min:w_max:100j, b_min:b_max:100j]
        pos_grid = np.vstack([ww.ravel(), bb.ravel()]) # 2 x (100 x 100)
        return pos_grid

    # define variables or constants
    X_test = np.linspace(min(Wafer1), max(Wafer1), num=N)
    x = tf.constant(X_test, dtype=tf.float32)
    w = tf.Variable(1.0, name='w')
    b = tf.Variable(1.9, name='b')
    
    # define pdf2 wrapper
    y_hypo = lin_model(x, w, b)
    ypdf_hypo = tf.py_function(pdf_wrapper, [y_hypo], tf.float32)
    
    
    # calculate real values from pdf1 and store them
    y_real = tf.constant(pdf1(X_test), dtype=tf.float32)
    
    # generate w,b grid
    grid_wb = generate_grid_data()
    
    # min error, and corresponding w and b
    min_error = None
    min_w = None
    min_b = None
    
    # start tensorflow
    init = tf.global_variables_initializer()
    s = tf.Session(graph=g)
    s.run(init)
    for i in range(N):
        current_w = grid_wb[0][i]
        current_b = grid_wb[1][i]
        
        # assign w and b
        opw = w.assign(current_w)
        s.run(opw)
        opb = w.assign(current_b)
        s.run(opb)
        
        print(s.run(w), s.run(b))
        
        y_predict = s.run(ypdf_hypo)
        cost = error_function(y_predict, y_real)
        err = s.run(cost)
        print("error:", err)
        print("direct value:", pdf2(X_test[0] + 1.9), "predict value:", y_predict[0])
        if min_error is None or min_error > err: # Record min err and its position
            min_error = err
            min_w = current_w
            min_b = current_b
    
    s.close()
    print("final:", min_w, min_b, min_error)

Instructions for updating:
Colocations handled automatically by placer.
-3.0 1.9
(10000,) (10000,)
error: 0.015154251
direct value: [5.45142323e-31] predict value: 0.0
-2.939394 1.9
(10000,) (10000,)
error: 0.01481881
direct value: [5.45142323e-31] predict value: 0.0
-2.878788 1.9
(10000,) (10000,)
error: 0.014473193
direct value: [5.45142323e-31] predict value: 0.0
-2.8181818 1.9
(10000,) (10000,)
error: 0.014117136
direct value: [5.45142323e-31] predict value: 0.0
-2.7575758 1.9
(10000,) (10000,)
error: 0.013750415
direct value: [5.45142323e-31] predict value: 0.0
-2.6969697 1.9
(10000,) (10000,)
error: 0.013372797
direct value: [5.45142323e-31] predict value: 0.0
-2.6363637 1.9
(10000,) (10000,)
error: 0.012983954
direct value: [5.45142323e-31] predict value: 0.0
-2.5757575 1.9
(10000,) (10000,)
error: 0.012583512
direct value: [5.45142323e-31] predict value: 0.0
-2.5151515 1.9
(10000,) (10000,)
error: 0.012171148
direct value: [5.45142323e-31] predict value: 0.0
-2.4545455 1.9
(100